In [1]:
import pandas as pd

In [2]:
result_file = "test_result_w_succ_diff_nodes_gru_2025_04_10_12_23.csv"

In [3]:
EPS = 1

In [4]:
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.000000
1,star_graph_n7,0.0,0.000000
2,star_graph_n7,0.0,0.000000
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,3.0,2.636848


In [5]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [6]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [7]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [8]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
1,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
2,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
3,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,star_graph_n7,3.0,2.636848,0.363152,0.131879,0.0,0.121051
...,...,...,...,...,...,...,...
5275,graph_powerlaw_cluster_graph_n7,3.0,3.418070,0.418070,0.174783,0.0,0.139357
5276,graph_powerlaw_cluster_graph_n7,4.0,4.206355,0.206355,0.042582,0.0,0.051589
5277,graph_powerlaw_cluster_graph_n7,3.0,3.065788,0.065788,0.004328,0.0,0.021929
5278,graph_powerlaw_cluster_graph_n7,4.0,4.170396,0.170396,0.029035,0.0,0.042599


In [9]:
df['RE'].sum()/len(df)

0.0839318790452348

In [10]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [11]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1701,133.057054,0.078223,0.113119
1,2.0,1642,191.921927,0.116883,0.108840
2,1.0,951,63.873698,0.067165,0.006470
3,4.0,641,39.795950,0.062084,0.104905
4,0.0,258,0.000000,0.000000,0.000000
5,5.0,86,14.203630,0.165158,0.828408
6,6.0,1,0.308062,0.308062,3.416490


# Group by Dataset

In [12]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,306.783303,0.078521,0.094189
1,graph_powerlaw_cluster_graph_n7,1350,135.020228,0.100015,0.110510
2,star_graph_n7,23,1.356791,0.058991,0.087148


# Group by Dataset W/O Rank 0

In [13]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3768,306.783303,0.081418,0.097663
1,graph_powerlaw_cluster_graph_n7,1246,135.020228,0.108363,0.119734
2,star_graph_n7,8,1.356791,0.169599,0.250550


# Group by Dataset and Rank

In [14]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("analysis_results/test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,422,66.907517,0.158549,0.188982
1,graph_powerlaw_cluster_graph_n7,1.0,375,31.550662,0.084135,0.009126
2,graph_powerlaw_cluster_graph_n7,3.0,321,28.820430,0.089783,0.149007
3,graph_powerlaw_cluster_graph_n7,4.0,105,5.181949,0.049352,0.074220
4,graph_powerlaw_cluster_graph_n7,0.0,104,0.000000,0.000000,0.000000
5,graph_powerlaw_cluster_graph_n7,5.0,22,2.251607,0.102346,0.317043
6,graph_powerlaw_cluster_graph_n7,6.0,1,0.308062,0.308062,3.416490
7,graph_random_regular_graph_n7_d4,3.0,1377,103.847079,0.075415,0.104642
8,graph_random_regular_graph_n7_d4,2.0,1220,125.014410,0.102471,0.081119
9,graph_random_regular_graph_n7_d4,1.0,575,31.789529,0.055286,0.004254
